# Universal Inpainting Notebook

Works with SDXL or SD 1.5, from Google Drive or fresh download

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os

# Check what models are available in Drive
models_available = []
if os.path.exists('/content/drive/MyDrive/inpainting_models/sdxl_inpainting/'):
    print("✅ SDXL found in Drive (5.5GB model)")
    models_available.append('sdxl')
if os.path.exists('/content/drive/MyDrive/inpainting_models/sd15_inpaint/'):
    print("✅ SD 1.5 found in Drive (2GB model)")
    models_available.append('sd15')

if not models_available:
    print("❌ No models found in Drive")
    print("\nYou can either:")
    print("1. Upload models to Drive first (recommended)")
    print("2. Download directly in Colab (will re-download each session)")

In [ ]:
# Install required packages
!pip install -q diffusers transformers accelerate safetensors pillow

In [ ]:
# Choose and load model
import torch
from PIL import Image
import numpy as np

# SELECT YOUR MODEL HERE
MODEL_CHOICE = "sdxl"  # Change to "sd15" for SD 1.5, or "download_sdxl" to download fresh

pipe = None

if MODEL_CHOICE == "sdxl" and 'sdxl' in models_available:
    print("Loading SDXL from Drive...")
    from diffusers import AutoPipelineForInpainting
    pipe = AutoPipelineForInpainting.from_pretrained(
        "/content/drive/MyDrive/inpainting_models/sdxl_inpainting",
        torch_dtype=torch.float16,
        variant="fp16",
        local_files_only=True
    ).to("cuda")
    
elif MODEL_CHOICE == "sd15" and 'sd15' in models_available:
    print("Loading SD 1.5 from Drive...")
    from diffusers import StableDiffusionInpaintPipeline
    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "/content/drive/MyDrive/inpainting_models/sd15_inpaint",
        torch_dtype=torch.float16,
        local_files_only=True
    ).to("cuda")
    
elif MODEL_CHOICE == "download_sdxl":
    print("Downloading SDXL (this will take time)...")
    from diffusers import AutoPipelineForInpainting
    pipe = AutoPipelineForInpainting.from_pretrained(
        "diffusers/stable-diffusion-xl-1.0-inpainting-0.1",
        torch_dtype=torch.float16,
        variant="fp16"
    ).to("cuda")
    
elif MODEL_CHOICE == "download_sd15":
    print("Downloading SD 1.5 (faster than SDXL)...")
    from diffusers import StableDiffusionInpaintPipeline
    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "runwayml/stable-diffusion-inpainting",
        torch_dtype=torch.float16
    ).to("cuda")
else:
    print(f"❌ Model '{MODEL_CHOICE}' not found in Drive!")
    print("Change MODEL_CHOICE to 'download_sdxl' or 'download_sd15' to download")

if pipe:
    # Optimize memory
    pipe.enable_xformers_memory_efficient_attention()
    if "sdxl" in MODEL_CHOICE:
        pipe.enable_model_cpu_offload()
    print("✅ Model loaded and ready!")

In [ ]:
# Upload your image and mask
from google.colab import files
import zipfile

print("Upload the zip file from room_removal_ultimate.py export")
uploaded = files.upload()

# Extract
zip_name = list(uploaded.keys())[0]
with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall('.')

# Load images
image = Image.open("image.png").convert("RGB")
mask = Image.open("mask.png").convert("L")

print(f"Loaded: Image {image.size}, Mask {mask.size}")

In [ ]:
# Run inpainting
if pipe is None:
    print("❌ No model loaded! Go back and load a model first.")
else:
    # Adjust parameters based on model
    if "sdxl" in MODEL_CHOICE:
        # SDXL parameters
        result = pipe(
            prompt="high quality interior, empty room, professional real estate photography, clean walls and floor",
            negative_prompt="furniture, objects, people, artifacts, blurry, distorted",
            image=image,
            mask_image=mask,
            num_inference_steps=50,
            strength=0.99,
            guidance_scale=8.0,
            height=image.height,
            width=image.width
        ).images[0]
    else:
        # SD 1.5 parameters
        result = pipe(
            prompt="empty room, clean walls, professional photography",
            negative_prompt="furniture, people, objects, low quality",
            image=image,
            mask_image=mask,
            num_inference_steps=50,
            guidance_scale=7.5,
            height=image.height,
            width=image.width
        ).images[0]
    
    print("✅ Inpainting complete!")

In [ ]:
# Display results
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(image)
axes[0].set_title('Original', fontsize=14)
axes[0].axis('off')

axes[1].imshow(mask, cmap='gray')
axes[1].set_title('Mask', fontsize=14)
axes[1].axis('off')

axes[2].imshow(result)
model_name = "SDXL" if "sdxl" in MODEL_CHOICE else "SD 1.5"
axes[2].set_title(f'{model_name} Result', fontsize=14)
axes[2].axis('off')

plt.tight_layout()
plt.show()

# Save and download
result.save("inpainted_result.png")
files.download("inpainted_result.png")

## Advanced Options

In [ ]:
# Multi-pass refinement for even better quality
def refine_result(image, mask, initial_result, num_passes=2):
    """Run multiple passes for refinement"""
    result = initial_result
    
    for pass_num in range(1, num_passes):
        print(f"Refinement pass {pass_num}...")
        
        # Use slightly different parameters for refinement
        result = pipe(
            prompt="high quality, photorealistic, professional",
            negative_prompt="artifacts, blurry",
            image=result,
            mask_image=mask,
            num_inference_steps=30,
            strength=0.5,  # Lower strength for refinement
            guidance_scale=5.0
        ).images[0]
    
    return result

# Uncomment to use:
# refined = refine_result(image, mask, result, num_passes=2)
# refined.save("refined_result.png")
# files.download("refined_result.png")

In [ ]:
# Batch processing multiple images
def process_batch(zip_path):
    """Process multiple image/mask pairs"""
    import zipfile
    import os
    
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall('batch')
    
    results = []
    for file in os.listdir('batch'):
        if file.startswith('image') and file.endswith('.png'):
            img = Image.open(f'batch/{file}').convert('RGB')
            mask_file = file.replace('image', 'mask')
            
            if os.path.exists(f'batch/{mask_file}'):
                msk = Image.open(f'batch/{mask_file}').convert('L')
                
                print(f"Processing {file}...")
                result = pipe(
                    prompt="empty room, clean",
                    image=img,
                    mask_image=msk,
                    num_inference_steps=50
                ).images[0]
                
                result.save(f'batch/result_{file}')
                results.append(f'batch/result_{file}')
    
    return results

# Uncomment to use:
# results = process_batch('your_batch.zip')